In [2]:
%matplotlib inline
from siphon.catalog import TDSCatalog

In [188]:
best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                      'Global_0p25deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p25deg/Best')
best_gfs.datasets

['Best GFS Quarter Degree Forecast Time Series']

In [189]:
best_ds = list(best_gfs.datasets.values())[0]
ncss = best_ds.subset()

In [190]:
query = ncss.query()

In [191]:
ncss.variables

{'Absolute_vorticity_isobaric',
 'Albedo_surface_Mixed_intervals_Average',
 'Apparent_temperature_height_above_ground',
 'Best_4_layer_Lifted_Index_surface',
 'Categorical_Freezing_Rain_surface',
 'Categorical_Freezing_Rain_surface_Mixed_intervals_Average',
 'Categorical_Ice_Pellets_surface',
 'Categorical_Ice_Pellets_surface_Mixed_intervals_Average',
 'Categorical_Rain_surface',
 'Categorical_Rain_surface_Mixed_intervals_Average',
 'Categorical_Snow_surface',
 'Categorical_Snow_surface_Mixed_intervals_Average',
 'Cloud_Work_Function_entire_atmosphere_single_layer_Mixed_intervals_Average',
 'Cloud_mixing_ratio_hybrid',
 'Cloud_mixing_ratio_isobaric',
 'Cloud_water_entire_atmosphere_single_layer',
 'Composite_reflectivity_entire_atmosphere',
 'Convective_Precipitation_Rate_surface_Mixed_intervals_Average',
 'Convective_available_potential_energy_pressure_difference_layer',
 'Convective_available_potential_energy_surface',
 'Convective_inhibition_pressure_difference_layer',
 'Convective_

In [203]:


from datetime import datetime
from datetime import timedelta
query.lonlat_box(north=-36.8, south=-36.9, west=174.7, east=174.8).time_range(datetime.utcnow(), 
                                                                              datetime.utcnow()+timedelta(15))
query.accept('netcdf4')
query.variables('u-component_of_wind_planetary_boundary',
                 'v-component_of_wind_planetary_boundary')



var=v-component_of_wind_planetary_boundary&var=u-component_of_wind_planetary_boundary&time_start=2022-12-22T02%3A32%3A39.377509&time_end=2023-01-06T02%3A32%3A39.377510&west=174.7&east=174.8&south=-36.9&north=-36.8&accept=netcdf4

In [204]:
from xarray.backends import NetCDF4DataStore
import xarray as xr

data = ncss.get_data(query)
ds = NetCDF4DataStore(data)
data = xr.open_dataset(ds)

list(data)

['u-component_of_wind_planetary_boundary',
 'v-component_of_wind_planetary_boundary',
 'LatLon_721X1440-0p13S-180p00E-2']

In [205]:
u_wind_3d = data['u-component_of_wind_planetary_boundary']
v_wind_3d = data['v-component_of_wind_planetary_boundary']

In [225]:
def get_wind_vector(data, time, lat, lon):
    u = data.sel(time=time,latitude=lat,longitude=lon, method='nearest')['u-component_of_wind_planetary_boundary'].metpy.unit_array.squeeze().to('knots')
    v = data.sel(time=time,latitude=lat,longitude=lon, method='nearest')['v-component_of_wind_planetary_boundary'].metpy.unit_array.squeeze().to('knots') 
    return [u,v]

In [233]:
class WeatherStation:
    def __init__(self):
        self.best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                      'Global_0p25deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p25deg/Best')
        self.ncss = best_ds.subset()
        self.query = self.ncss.query()
        self.time_range = None
        self.data = None
    
    def load_data(self, north, south, east, west, 
                  start_date=datetime.utcnow(), 
                  end_date=datetime.utcnow() + timedelta(15)):
        # build the query
        self.time_range = (start_date, end_date)
        self.query.lonlat_box(north=north, south=south, west=west, east=east).time_range(datetime.utcnow(), 
                                                                              datetime.utcnow()+timedelta(15))
        self.query.accept('netcdf4')
        self.query.variables('u-component_of_wind_planetary_boundary',
                             'v-component_of_wind_planetary_boundary')
        
        # load the data from server
        data = ncss.get_data(query)
        ds = NetCDF4DataStore(data)
        self.data = xr.open_dataset(ds)
        
    def get_wind_vector(self, time, lat, lon):
        # gets the wind vector at location lat/lon at time in knots
        if self.data is None:
            raise Exception('Load base data before calling this method!')
        u = data.sel(time=time,latitude=lat,longitude=lon, method='nearest')['u-component_of_wind_planetary_boundary'].metpy.unit_array.squeeze().to('knots')
        v = data.sel(time=time,latitude=lat,longitude=lon, method='nearest')['v-component_of_wind_planetary_boundary'].metpy.unit_array.squeeze().to('knots') 
        return [u,v]

In [234]:
ws = WeatherStation()

In [236]:
ws.load_data(north=-35, south=-38,west=173, east=175, start_date=datetime.utcnow(), end_date=datetime.utcnow() + timedelta(1))

In [238]:
ws.get_wind_vector(datetime.utcnow()+timedelta(1), -36, 174)

[11.17718739849445 <Unit('knot')>, -5.871412306058485 <Unit('knot')>]

In [207]:


# Helper function for finding proper time variable
def find_time_var(var, time_basename='time'):
    for coord_name in var.coords:
        if coord_name.startswith(time_basename):
            return var.coords[coord_name]
    raise ValueError('No time variable found for ' + var.name)



In [209]:
time_1d = find_time_var(u_wind_3d)
lat_1d = data['latitude']
lon_1d = data['longitude']
time_1d

<xarray.DataArray 'time' (time: 89)>
array(['2022-12-22T03:00:00.000000000', '2022-12-22T06:00:00.000000000',
       '2022-12-22T09:00:00.000000000', '2022-12-22T12:00:00.000000000',
       '2022-12-22T15:00:00.000000000', '2022-12-22T18:00:00.000000000',
       '2022-12-22T21:00:00.000000000', '2022-12-23T00:00:00.000000000',
       '2022-12-23T03:00:00.000000000', '2022-12-23T06:00:00.000000000',
       '2022-12-23T09:00:00.000000000', '2022-12-23T12:00:00.000000000',
       '2022-12-23T15:00:00.000000000', '2022-12-23T18:00:00.000000000',
       '2022-12-23T21:00:00.000000000', '2022-12-24T00:00:00.000000000',
       '2022-12-24T03:00:00.000000000', '2022-12-24T06:00:00.000000000',
       '2022-12-24T09:00:00.000000000', '2022-12-24T12:00:00.000000000',
       '2022-12-24T15:00:00.000000000', '2022-12-24T18:00:00.000000000',
       '2022-12-24T21:00:00.000000000', '2022-12-25T00:00:00.000000000',
       '2022-12-25T03:00:00.000000000', '2022-12-25T06:00:00.000000000',
       '2022-12-25T09:00:00.000000000', '2022-12-25T12:00:00.000000000',
       '2022-12-25T15:00:00.000000000', '2022-12-25T18:00:00.000000000',
       '2022-12-25T21:00:00.000000000', '2022-12-26T00:00:00.000000000',
       '2022-12-26T03:00:00.000000000', '2022-12-26T06:00:00.000000000',
       '2022-12-26T09:00:00.000000000', '2022-12-26T12:00:00.000000000',
       '2022-12-26T15:00:00.000000000', '2022-12-26T18:00:00.000000000',
       '2022-12-26T21:00:00.000000000', '2022-12-27T00:00:00.000000000',
       '2022-12-27T03:00:00.000000000', '2022-12-27T06:00:00.000000000',
       '2022-12-27T09:00:00.000000000', '2022-12-27T12:00:00.000000000',
       '2022-12-27T15:00:00.000000000', '2022-12-27T18:00:00.000000000',
       '2022-12-27T21:00:00.000000000', '2022-12-28T00:00:00.000000000',
       '2022-12-28T03:00:00.000000000', '2022-12-28T06:00:00.000000000',
       '2022-12-28T09:00:00.000000000', '2022-12-28T12:00:00.000000000',
       '2022-12-28T15:00:00.000000000', '2022-12-28T18:00:00.000000000',
       '2022-12-28T21:00:00.000000000', '2022-12-29T00:00:00.000000000',
       '2022-12-29T03:00:00.000000000', '2022-12-29T06:00:00.000000000',
       '2022-12-29T09:00:00.000000000', '2022-12-29T12:00:00.000000000',
       '2022-12-29T15:00:00.000000000', '2022-12-29T18:00:00.000000000',
       '2022-12-29T21:00:00.000000000', '2022-12-30T00:00:00.000000000',
       '2022-12-30T03:00:00.000000000', '2022-12-30T06:00:00.000000000',
       '2022-12-30T09:00:00.000000000', '2022-12-30T12:00:00.000000000',
       '2022-12-30T15:00:00.000000000', '2022-12-30T18:00:00.000000000',
       '2022-12-30T21:00:00.000000000', '2022-12-31T00:00:00.000000000',
       '2022-12-31T03:00:00.000000000', '2022-12-31T06:00:00.000000000',
       '2022-12-31T09:00:00.000000000', '2022-12-31T12:00:00.000000000',
       '2022-12-31T15:00:00.000000000', '2022-12-31T18:00:00.000000000',
       '2023-01-01T06:00:00.000000000', '2023-01-01T18:00:00.000000000',
       '2023-01-02T06:00:00.000000000', '2023-01-02T18:00:00.000000000',
       '2023-01-03T06:00:00.000000000', '2023-01-03T18:00:00.000000000',
       '2023-01-04T06:00:00.000000000', '2023-01-04T18:00:00.000000000',
       '2023-01-05T06:00:00.000000000', '2023-01-05T18:00:00.000000000',
       '2023-01-06T06:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
    reftime  (time) datetime64[ns] ...
  * time     (time) datetime64[ns] 2022-12-22T03:00:00 ... 2023-01-06T06:00:00
Attributes:
    standard_name:  time
    long_name:      GRIB forecast or observation time

In [65]:
import numpy as np
from netCDF4 import num2date
from metpy.units import units

# Reduce the dimensions of the data and get as an array with units
u_wind_2d = u_wind_3d.metpy.unit_array.squeeze()
v_wind_2d = v_wind_3d.metpy.unit_array.squeeze()
# Combine latitude and longitudes 
lon_2d, lat_2d = np.meshgrid(lon_1d, lat_1d)

In [66]:
lat_1d

<xarray.DataArray 'latitude' (latitude: 2)>
array([-36.75, -37.  ], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 -36.75 -37.0
Attributes:
    units:          degrees_north
    standard_name:  latitude

In [67]:
lon_2d.flatten()

array([174.75, 174.75], dtype=float32)

In [68]:
lat_2d.flatten()

array([-36.75, -37.  ], dtype=float32)

In [147]:
u_wind_3d

<xarray.DataArray 'u-component_of_wind_planetary_boundary' (time: 1,
                                                            latitude: 2,
                                                            longitude: 1)>
array([[[1.867207],
        [2.867207]]], dtype=float32)
Coordinates:
    reftime    (time) datetime64[ns] ...
  * latitude   (latitude) float32 -36.75 -37.0
  * time       (time) datetime64[ns] 2022-12-22
  * longitude  (longitude) float32 174.8
Attributes:
    long_name:                       u-component of wind @ Planetary Boundary...
    units:                           m/s
    Grib_Variable_Id:                VAR_7-0--1-2_L220
    Grib2_Parameter:                 [0 2 2]
    Grib2_Parameter_Discipline:      Meteorological products
    Grib2_Parameter_Category:        Momentum
    Grib2_Parameter_Name:            u-component of wind
    Grib2_Level_Type:                220
    Grib2_Level_Desc:                Planetary Boundary Layer
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1
    grid_mapping:                    LatLon_721X1440-0p13S-180p00E-2

In [163]:
u = u_wind_3d.sel(latitude=-36,longitude=174, method='nearest').metpy.unit_array.squeeze().to('knots')
v = v_wind_3d.sel(latitude=-36,longitude=174, method='nearest').metpy.unit_array.squeeze().to('knots')

In [175]:
v.magnitude

-7.647977791903601

In [176]:
np.math.atan2(v.magnitude,u.magnitude)

-1.127692658595143

In [77]:
360-64.61202992538894

295.3879700746111

In [177]:
from scipy.interpolate import interp2d
# IMOCA 60 boat speed
x = [5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35,
     5, 10, 15, 20, 25, 30, 35]
y = [30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
     30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180]
y = np.array(y)*np.pi/(180.0)
z = [4.93,7.88,7.44,7.27,6.80,6.73,6.85,
     6.68,9.78,9.63,9.78,9.52,9.51,9.72,
     7.76,10.74,10.67,10.95,10.80,10.92,11.22,
     8.48,11.40,11.52,11.95,11.93,12.17,12.56,
     9.01,11.99,12.42,13.16,13.42,13.85,14.36,
     9.27,12.58,13.55,14.72,15.34,16.03,16.70,
     9.33,13.12,14.91,16.67,17.77,18.72,19.57,
     9.06,13.54,16.19,18.32,19.62,20.65,21.57,
     9.11,13.66,17.45,20.04,21.56,22.67,23.68,
     9.03,13.95,18.76,21.88,23.72,24.88,25.98,
     8.62,14.04,18.64,22.51,25.17,26.23,27.35,
     7.84,13.91,19.28,22.95,25.24,26.39,27.54,
     6.94,13.38,19.41,23.27,25.65,26.82,27.98,
     5.93,12.31,17.56,21.20,23.52,24.54,25.60,
     5.28,11.40,16.01,19.23,21.30,22.24,23.20,
     4.43,9.80,14.38,17.25,19.01,19.88,20.74]
f = interp2d(x,y,z, kind='cubic')

/home/justin/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=14,16 m=112 fp=1864.421137 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [179]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))

In [184]:
angle = np.pi - angle_between([1,0], [u.magnitude,v.magnitude])

In [185]:
wind_speed = np.sqrt(u.magnitude**2 + v.magnitude**2)

In [186]:
f(wind_speed, angle)

array([11.54191745])

In [245]:
import math
EARTH_RADIUS = 6371.00
def add_distance(lat, lon, bearing, distance):
    # Bearing in radians
    # Distance in km
    
    # convert Latitude and Longitude
    # into radians for calculation
    latitude = math.radians(lat)
    longitute = math.radians(lon)

    # calculate next latitude
    next_latitude = math.asin(math.sin(latitude) *
                    math.cos(distance/EARTH_RADIUS) +
                    math.cos(latitude) *
                    math.sin(distance/EARTH_RADIUS) *
                    math.cos(bearing))

    # calculate next longitude
    next_longitude = longitute + (math.atan2(math.sin(bearing) *
                                             math.sin(distance/EARTH_RADIUS) *
                                             math.cos(latitude),
                                             math.cos(distance/EARTH_RADIUS) -
                                             math.sin(latitude) *
                                             math.sin(next_latitude)
                                            )
                                 )

    # convert points into decimal degrees
    new_lat = math.degrees(next_latitude)
    new_lon = math.degrees(next_longitude)
    return [new_lat, new_lon]

In [246]:
add_distance(lat=-34, lon=174, bearing=3*np.pi/2, distance=100)

[-33.99523959635117, 172.91526266298578]

In [ ]:
import math

class Boat:
    def __init__(self, lat, lon, time=datetime.utcnow()):
        self.vel_over_water = self._init_vel_over_water()
        self.lat = lat
        self.lon = lon
        self.current_time = time
        # standard earth radius
        self.
    

    
    def _init_vel_over_water():
        # IMOCA 60 boat speed
        x = [5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35,
             5, 10, 15, 20, 25, 30, 35]
        y = [30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,
             30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180]
        y = np.array(y)*np.pi/(180.0)
        z = [4.93,7.88,7.44,7.27,6.80,6.73,6.85,
             6.68,9.78,9.63,9.78,9.52,9.51,9.72,
             7.76,10.74,10.67,10.95,10.80,10.92,11.22,
             8.48,11.40,11.52,11.95,11.93,12.17,12.56,
             9.01,11.99,12.42,13.16,13.42,13.85,14.36,
             9.27,12.58,13.55,14.72,15.34,16.03,16.70,
             9.33,13.12,14.91,16.67,17.77,18.72,19.57,
             9.06,13.54,16.19,18.32,19.62,20.65,21.57,
             9.11,13.66,17.45,20.04,21.56,22.67,23.68,
             9.03,13.95,18.76,21.88,23.72,24.88,25.98,
             8.62,14.04,18.64,22.51,25.17,26.23,27.35,
             7.84,13.91,19.28,22.95,25.24,26.39,27.54,
             6.94,13.38,19.41,23.27,25.65,26.82,27.98,
             5.93,12.31,17.56,21.20,23.52,24.54,25.60,
             5.28,11.40,16.01,19.23,21.30,22.24,23.20,
             4.43,9.80,14.38,17.25,19.01,19.88,20.74]
        f = interp2d(x,y,z, kind='cubic')
        return f